In [1]:
import numpy as np
import pandas as pd

In [3]:
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import OrdinalEncoder

In [4]:
df =  pd.read_csv('covid_toy.csv')

In [5]:
df.head()

,age,gender,fever,cough,city,has_covid
0,60,Male,103.0,Mild,Kolkata,No
1,27,Male,100.0,Mild,Delhi,Yes
2,42,Male,101.0,Mild,Delhi,No
3,31,Female,98.0,Mild,Kolkata,No
4,65,Female,101.0,Mild,Mumbai,No


In [8]:
df.isnull().sum()

age           0
gender        0
fever        10
cough         0
city          0
has_covid     0
dtype: int64

In [10]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(df.iloc[:, 0:5], df.iloc[:, -1], test_size=0.2)

## 1. Mannual Way of preprocessing without using column transformer

In [11]:
si = SimpleImputer()

X_train_fever = si.fit_transform(X_train[['fever']])
X_test_fever = si.fit_transform(X_test[['fever']])

X_train_fever

array([[102.        ],
       [ 99.        ],
       [100.84722222],
       [100.84722222],
       [ 98.        ],
       [100.        ],
       [ 98.        ],
       [101.        ],
       [104.        ],
       [101.        ],
       [103.        ],
       [100.84722222],
       [102.        ],
       [ 99.        ],
       [104.        ],
       [102.        ],
       [103.        ],
       [101.        ],
       [ 98.        ],
       [101.        ],
       [101.        ],
       [100.        ],
       [100.        ],
       [104.        ],
       [101.        ],
       [104.        ],
       [100.        ],
       [ 98.        ],
       [ 98.        ],
       [102.        ],
       [100.84722222],
       [100.84722222],
       [101.        ],
       [ 98.        ],
       [ 98.        ],
       [ 98.        ],
       [100.        ],
       [100.        ],
       [ 99.        ],
       [102.        ],
       [ 99.        ],
       [ 98.        ],
       [100.        ],
       [100

In [16]:
oe = OrdinalEncoder(categories=[['Mild', 'Strong']])

X_train_cough = oe.fit_transform(X_train[['cough']])
X_test_cough = oe.fit_transform(X_test[['cough']])

In [18]:
ohe = OneHotEncoder(drop='first', sparse_output=False)

X_train_gender_city = ohe.fit_transform(X_train[['gender', 'city']])
X_test_gender_city = ohe.fit_transform(X_test[['gender', 'city']])

In [19]:
X_train_age = X_train[['age']]
X_test_age = X_test[['age']]

In [20]:
X_train_transformed = np.concatenate((X_train_age, X_train_gender_city, X_train_fever, X_train_cough), axis=1)
X_test_transformed = np.concatenate((X_test_age, X_test_gender_city, X_test_fever, X_test_cough), axis=1)

In [23]:
X_train_transformed.shape

(80, 7)

In [24]:
X_train_transformed

array([[ 33.        ,   0.        ,   1.        ,   0.        ,
          0.        , 102.        ,   1.        ],
       [ 22.        ,   0.        ,   0.        ,   0.        ,
          0.        ,  99.        ,   0.        ],
       [ 71.        ,   1.        ,   0.        ,   1.        ,
          0.        , 100.84722222,   1.        ],
       [ 82.        ,   1.        ,   0.        ,   1.        ,
          0.        , 100.84722222,   0.        ],
       [ 26.        ,   0.        ,   0.        ,   1.        ,
          0.        ,  98.        ,   0.        ],
       [ 27.        ,   1.        ,   0.        ,   1.        ,
          0.        , 100.        ,   0.        ],
       [ 10.        ,   0.        ,   0.        ,   1.        ,
          0.        ,  98.        ,   1.        ],
       [ 65.        ,   0.        ,   0.        ,   0.        ,
          1.        , 101.        ,   0.        ],
       [ 18.        ,   0.        ,   0.        ,   0.        ,
          0.    

In [32]:
X_train_transformed_df = pd.DataFrame(X_train_transformed)
X_train_transformed_df

,0,1,2,3,4,5,6
0,33.0,0.0,1.0,0.0,0.0,102.000000,1.0
1,22.0,0.0,0.0,0.0,0.0,99.000000,0.0
2,71.0,1.0,0.0,1.0,0.0,100.847222,1.0
3,82.0,1.0,0.0,1.0,0.0,100.847222,0.0
4,26.0,0.0,0.0,1.0,0.0,98.000000,0.0
...,...,...,...,...,...,...,...
75,64.0,0.0,0.0,0.0,0.0,98.000000,0.0
76,24.0,1.0,0.0,1.0,0.0,98.000000,0.0
77,64.0,0.0,1.0,0.0,0.0,101.000000,0.0
78,34.0,0.0,1.0,0.0,0.0,104.000000,1.0


## 2. Preprocessing Using ColumnTransformer

In [33]:
from sklearn.compose import ColumnTransformer

In [41]:
transformer = ColumnTransformer(transformers=[
    ('tf1', SimpleImputer(), ['fever']),
    ('tf2', OrdinalEncoder(categories=[['Mild', 'Strong']]), ['cough']),
    ('tf3', OneHotEncoder(drop='first', sparse_output=False), ['gender', 'city'])
], remainder='passthrough')

In [42]:
transformed_X_train = transformer.fit_transform(X_train)
transformed_X_test = transformer.fit_transform(X_test)

In [45]:
transformed_X_train.shape

(80, 7)